In [0]:
from pyspark.sql.types import StructType,StructField,IntegerType,StringType,DoubleType,DateType,FloatType
from pyspark.sql.functions import col ,lit,concat,to_timestamp,concat_ws

In [0]:
%run "../includes/configuration"


In [0]:
%run "../includes/Common_functions"

In [0]:
# Create a text widget in Databricks with a default value "2021-03-21"
dbutils.widgets.text("p_file_date","2021-03-21")

# Retrieve the value entered in the widget and store it in a variable
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
##### Step 1 - Read the JSON file using the spark dataframe reader

results_schema = StructType(fields=[StructField("resultId", IntegerType(), False),
                                    StructField("raceId", IntegerType(), True),
                                    StructField("driverId", IntegerType(), True),
                                    StructField("constructorId", IntegerType(), True),
                                    StructField("number", IntegerType(), True),
                                    StructField("grid", IntegerType(), True),
                                    StructField("position", IntegerType(), True),
                                    StructField("positionText", StringType(), True),
                                    StructField("positionOrder", IntegerType(), True),
                                    StructField("points", FloatType(), True),
                                    StructField("laps", IntegerType(), True),
                                    StructField("time", StringType(), True),
                                    StructField("milliseconds", IntegerType(), True),
                                    StructField("fastestLap", IntegerType(), True),
                                    StructField("rank", IntegerType(), True),
                                    StructField("fastestLapTime", StringType(), True),
                                    StructField("fastestLapSpeed", FloatType(), True),
                                    StructField("statusId", StringType(), True)])

results_df = spark.read \
.schema(results_schema) \
.json(f"{raw_folder_path}/{v_file_date}/results.json")


In [0]:
# Step 2 - Rename columns and add new columns

# 1- Rename columns and add new columns
results_with_columns_df = results_df.withColumnRenamed("resultId", "result_id") \
                                    .withColumnRenamed("raceId", "race_id") \
                                    .withColumnRenamed("driverId", "driver_id") \
                                    .withColumnRenamed("constructorId", "constructor_id") \
                                    .withColumnRenamed("positionText", "position_text") \
                                    .withColumnRenamed("positionOrder", "position_order") \
                                    .withColumnRenamed("fastestLap", "fastest_lap") \
                                    .withColumnRenamed("fastestLapTime", "fastest_lap_time") \
                                    .withColumnRenamed("fastestLapSpeed", "fastest_lap_speed") \
                                    .withColumn("file_date", lit(v_file_date))

# 2- ingestion date added
results_with_ingestion_date_df = add_ingestion_date(results_with_columns_df)

In [0]:
# Step 3 - Drop the unwanted column
results_final_df = results_with_ingestion_date_df.drop(col("statusId"))


In [0]:
# Step 4 - Write to output to processed container in parquet format
##Method 1
# for race_id_list in results_final_df.select("race_id").distinct().collect():
#     if (spark._jsparkSession.catalog().tableExists("f1_processed.results")):
#         spark.sql(f"ALTER TABLE f1_processed.results DROP IF EXISTS PARTITION (race_id={race_id_list.race_id})")
        
# results_final_df.write.mode("append").partitionBy('race_id').format("parquet").saveAsTable("results_final_df.results")

In [0]:
# Methode 2
# overwrite_partition(results_final_df,'f1_processed', 'results', 'race_id')

In [0]:
results_deduped_df = results_final_df.dropDuplicates(['race_id', 'driver_id'])

In [0]:
merge_condition = "tgt.result_id = src.result_id AND tgt.race_id = src.race_id"
merge_delta_data(results_deduped_df, 'f1_processed', 'results', processed_folder_path, merge_condition, 'race_id')

In [0]:
%sql
SELECT race_id, count(1) FROM f1_processed.results GROUP BY race_id

race_id,count(1)
1053,20
1052,20
